In [1]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, GRU
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


2024-12-01 18:31:57.083501: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-01 18:31:57.209802: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [76]:
# Load the data sets and combine them into a single usable dataframe called data series
data_series = pd.read_csv('./data/data_avg_temp.csv')

# rename columns to be more descriptive
data_series.columns = ['date', 'avg_temp', 'avg_temp_anomaly']
data_series.drop(columns=['avg_temp_anomaly'], inplace=True)


# append to the dataframe
data_cool_degree = pd.read_csv('./data/data_cool_degree_days.csv')
data_heat_degree = pd.read_csv('./data/data_heat_degree_days.csv')
data_max_temp = pd.read_csv('./data/data_max_temp.csv')
data_min_temp = pd.read_csv('./data/data_min_temp.csv')
data_palmer_z = pd.read_csv('./data/data_palmer_z.csv')
data_pdsi = pd.read_csv('./data/data_pdsi.csv')
data_phdi = pd.read_csv('./data/data_phdi.csv')
data_pmdi = pd.read_csv('./data/data_pmdi.csv')
data_precipitation = pd.read_csv('./data/data_precipitation.csv')
Death_New_Mexico = pd.read_csv('./data/Death_New_Mexico.csv')
#read in the data for deaths here , make sure empty columns are not there . Use the drop command to just remove the columns. 
Death_New_Mexico.drop(columns = ['Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11'], inplace=True)
Death_New_Mexico.drop([0, 1], inplace=True)
Death_New_Mexico['Deaths'] = Death_New_Mexico['Deaths'].str.replace(',', '')
print(Death_New_Mexico.head())




# Append columns to the data series
data_series['cool_degree_days'] = data_cool_degree['Value'] 
# data_series['cool_degree_days_anomaly'] = data_cool_degree['Anomaly']

data_series['heat_degree_days'] = data_heat_degree['Value']
# data_series['heat_degree_days_anomaly'] = data_heat_degree['Anomaly']

data_series['max_temp'] = data_max_temp['Value']
# data_series['max_temp_anomaly'] = data_max_temp['Anomaly']

data_series['min_temp'] = data_min_temp['Value']
# data_series['min_temp_anomaly'] = data_min_temp['Anomaly']

data_series['palmer_z'] = data_palmer_z['Value']
# data_series['palmer_z_anomaly'] = data_palmer_z['Anomaly']

data_series['pdsi'] = data_pdsi['Value']
# data_series['pdsi_anomaly'] = data_pdsi['Anomaly']

data_series['phdi'] = data_phdi['Value']
# data_series['phdi_anomaly'] = data_phdi['Anomaly']

data_series['pmdi'] = data_pmdi['Value']
# data_series['pmdi_anomaly'] = data_pmdi['Anomaly']

data_series['precipitation'] = data_precipitation['Value']
# data_series['precipitation_anomaly'] = data_precipitation['Anomaly']




     Year Population  Births Deaths
2  2017.0  2,102,521  23,708  18672
3  2016.0  2,103,586  24,503  18260
4  2015.0  2,099,856  25,730  17687
5  2014.0  2,098,381  25,985  17564
6  2013.0  2,095,156  26,242  16780


In [77]:
# Convert dataframe to numpy array
avg_temp = data_series['avg_temp'].to_numpy()
data_series.drop(columns=['avg_temp'], inplace=True)

## All our data as a numpy except the death_rate
x = data_series.to_numpy()

# Expand first dim
x = np.expand_dims(x, axis=0)
avg_temp = np.expand_dims(avg_temp, axis=0)

##Numpy array for the deathrate 
death_rate = Death_New_Mexico['Deaths'].to_numpy()
 
death_rate = np.array(death_rate, dtype=float)

# death_rate = np.expand_dims(death_rate, axis=0)
years = Death_New_Mexico['Year'].to_numpy()
# years = np.expand_dims(years, axis=0) 

years = np.expand_dims(years, axis=1)
death_rate = np.expand_dims(death_rate, axis=1)


In [ ]:
##death_rate Dense Network
X_train, X_test, y_train, y_test = train_test_split(years, death_rate, test_size=0.2,shuffle=False) ##split the data into training and testing data
                                               
model = Sequential([ ##Dense nueral network, consisting of the two inputs year and month, 
        Dense(5, activation='relu', input_shape=(X_train.shape[1],)),  # input_shape=(2,)
        Dense(1)  # Output layer
    ]) ##defines structure of model  ##Use this to train death data to then predict years for the past. x_train will be year, and y_train will be death rate. x_test and y_test split

model.compile(optimizer='adam', loss='mse') ##
model.fit(X_train, y_train, epochs=1000, validation_data=(X_test, y_test),verbose=0) ##fit the model to the

years_to_pred_past = np.linspace(1895, 1959, 65) ##years to predict
years_in_future = np.linspace(2018,2023, 6)


death_pred_past = model.predict(years_to_pred_past) ##predict the death rate for the years 1895 to 1960
death_pred_future = model.predict(years_in_future) ##predict the death rate for the years 2017 to 2020


years = np.append(years_to_pred_past, np.flip(years)) ##append the years to the years array
years = np.append(years, years_in_future) ##append the years to the years array

death_rate = np.append(death_pred_past, np.flip(death_rate)) ##append the death rate to the death rate array
death_rate = np.append(death_rate, death_pred_future) ##append the death rate to the death rate array

print(years)
print(death_rate)

# 0. Split the death rate data over 12 months so that it has the same length as the other parameters. 
# 1. Add the death rate to the parameter list 
# 2. Use it to train the GRU model
# 3. Predict the death rate forward in time using the dense models (like the other parameters)
# 4. Use the predicted death rate to train the GRU model
# Optionally make other parameter predictions and use them like the death rate
# Optionally also improve this model above to actually get better data. 


/home/frederik/miniconda3/envs/ml_new_mexico/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
[1895. 1896. 1897. 1898. 1899. 1900. 1901. 1902. 1903. 1904. 1905. 1906.
 1907. 1908. 1909. 1910. 1911. 1912. 1913. 1914. 1915. 1916. 1917. 1918.
 1919. 1920. 1921. 1922. 1923. 1924. 1925. 1926. 1927. 1928. 1929. 1930.
 1931. 1932. 1933. 1934. 1935. 1936. 1937. 1938. 1939. 1940. 1941. 1942.
 1943. 1944. 1945. 1946. 1947. 1948. 1949. 1950. 1951. 1952. 1953. 1954.
 1955. 1956. 1957. 1958. 1959. 1960. 1961. 1962. 1963. 1964. 1965. 1966.
 1967. 1968. 1969. 1970. 1971. 1972. 1973. 1974. 1975. 1976. 1977. 1978.
 1979. 1980. 1981. 1982. 1983. 1984. 1985. 1986. 1987. 1988. 1989. 1990.
 1991. 1992. 1993. 1994. 1995. 1996. 1997. 1998. 1999. 2000. 2001. 2002.
 2003. 2004. 2005. 2006. 2007. 2008. 2009. 2010. 2011. 2012. 2013. 2014.
 2015. 2016. 2017. 2018. 2019. 2020.]
[10446.19433594 10451.70410156 10457.21289062 10462.72167969
 10468.23046875 10473.74023438 10479.24902344 10484.7578125
 10490.26660156 10495.77636719 1050

In [5]:
# Train the GRU model 
model_gru = Sequential( 
    [
        GRU(20, activation='relu', input_shape=(x.shape[1], x.shape[2])), 
        Dense(1) 
    ]
)

model_gru.compile(optimizer='adam', loss='mse') 
model_gru.fit(x, avg_temp, epochs=300, batch_size=1, verbose=0) 


/Users/marienasmiscellanous/miniconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
# Predict the temperature by regressing the other features and then predicting average temperature
parameters = ['cool_degree_days', 'heat_degree_days', 'max_temp', 
              'min_temp', 'palmer_z', 'pdsi',
              'phdi', 'pmdi', 'precipitation']
# parameters = ['cool_degree_days']
models = {}
scalers = {}

date = data_series['date'].to_numpy()
date_splice = np.zeros((len(date), 2))

param_models = []

for i, element in enumerate(date):
    # print(element)
    # Splice this string to get year and month
    date_splice[i,0] = int(str(element)[0:4]) ##year 
    date_splice[i,1] = int(str(element)[5:7]) ##month
## split data into 12 by year to get months 
for param in parameters: ##for every parameter, we train the model. We do this 9 times because we have 9 parameters 
    print(f"PARAM", param)
    y = data_series[param].to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(date_splice, y, test_size=0.2, shuffle=True) ##splits data to training and testing... use this same line to generate x_test, then use it as your input to the model

    model = Sequential([ ##Dense nueral network, consisting of the two inputs year and month, 
        Dense(5, activation='relu', input_shape=(X_train.shape[1],)),  # input_shape=(2,)
        Dense(1)  # Output layer
    ]) ##defines structure of model  ##Use this to train death data to then predict years for the past. x_train will be year, and y_train will be death rate. x_test and y_test split

    model.compile(optimizer='adam', loss='mse') ##
    model.fit(X_train, y_train, epochs=300, batch_size=16, validation_data=(X_test, y_test),verbose=0) ##training loop 

    # Save model
    param_models.append(model) ##collect all the models into param_model


#Write a script that runs 1000 times (Monte Carlo), and average out the simulations over time, and plot them together. 

PARAM cool_degree_days
PARAM heat_degree_days
PARAM max_temp
PARAM min_temp
PARAM palmer_z
PARAM pdsi
PARAM phdi
PARAM pmdi
PARAM precipitation


In [ ]:
# Prediction of the average temperature
# Using the trained models to predict the parameters and then use those predictions as input for the GRU model

# Predict the parameters for the next 60 year * 12 months (720 datapoints)

# Get dates for prediction (202401-202812)
years = 60
date_predict = np.zeros((years*12, 2)) ##for year and month prediction 
for i in range(years*12):
    date_predict[i, 0] = 2024 + i // 12 ##year 
    date_predict[i, 1] = 1 + i % 12 ##month


param_predicts = np.zeros((len(parameters),years*12))

#  Predict the parameters.
for k, model in enumerate(param_models):
    for i in range(years*12):
        answer = model.predict(date_predict[i].reshape(1, -1)) ##produces predicted parameter output
        param_predicts[k, i] = answer

yearmonth_vec = np.zeros((years*12,1))
for i in range(years*12):
    yearmonth_vec[i] = date_predict[i, 0] * 100 + date_predict[i, 1] ##combined year and month value

params_predicts = np.append(yearmonth_vec, param_predicts.T, axis=1, verbose=0)

print(params_predicts)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


/var/folders/fk/60dn0cy51jx_fj73880s1s8m0000gn/T/ipykernel_75826/14198915.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  param_predicts[k, i] = answer


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━

In [ ]:
for i in range(years*12):
    print(f"Year: {int(params_predicts[i,0]//100)} Month: {int(params_predicts[i,0]%100)}")
    print(f"Predicted parameters:")
    for j, param in enumerate(parameters):
        print(f"{param}: {params_predicts[i,j+1]}")

In [ ]:

# Use predicted params for GRU input to predict avg_temp
predictions = []

for i in range(years*12):
    new_timestep = params_predicts[i,:]
    new_timestep = np.expand_dims(new_timestep, axis=0)
    x = np.append(x[:,1:,:], np.expand_dims(new_timestep, axis=0), axis=1)

    # x = np.expand_dims(params_predicts[i, :], axis=0)
    avg_temp_predict = model_gru.predict(x) ##Here's the money 
    predictions.append(avg_temp_predict)


print(predictions)


In [ ]:
# plot the predictions
predictions = np.array(predictions).reshape(-1)
plt.plot(yearmonth_vec[2:], predictions[2:], label='Predicted')


In [1]:
year = int(input("What year would you like to know the average temperature in New Mexico for?"))
month = input("What month in that year are you inquiring about?")

##standardize the inputs 
if ("jan" in month.lower() or '1' in month.lower()):
    month_i = 1
elif ("feb" in month.lower() or '2' in month.lower()):
    month_i = 2
elif ("mar" in month.lower() or '3' in month.lower()):
    month_i = 3
elif ("apr" in month.lower() or '4' in month.lower()):
    month_i = 4
elif ("may" in month.lower() or '5' in month.lower()):
    month_i = 5
elif ("jun" in month.lower() or '6' in month.lower()):
    month_i = 6
elif ("jul" in month.lower() or '7' in month.lower()):
    month_i = 7
elif ("aug" in month.lower() or '8' in month.lower()):
    month_i = 8
elif ("sep" in month.lower() or '9' in month.lower()):
    month_i = 9
elif ("oct" in month.lower() or '10' in month.lower()):
    month_i = 10
elif ("nov" in month.lower() or '11' in month.lower()):
    month_i = 11
elif ("dec" in month.lower() or '12' in month.lower()):
    month_i = 12
year_i = year-2024 
index = (year_i * 12) + month_i - 1
print(f"Prediction for the month {month} in the year {year} is: {predictions[index]}")


What year would you like to know the average temperature in New Mexico for? 2068
What month in that year are you inquiring about? March


NameError: name 'predictions' is not defined